In [119]:
import re
import pandas as pd

In [120]:
txt = """lr : 2.238721138568338e-05
Epoch: 0
Train : Counter({'tn': tensor(26938), 'tp': tensor(25957), 'fn': tensor(3269), 'fp': tensor(1112)})
Validation : Counter({'tn': tensor(8900), 'tp': tensor(8673), 'fn': tensor(1120), 'fp': tensor(399)})
Test : Counter({'tn': tensor(9081), 'tp': tensor(8571), 'fn': tensor(1080), 'fp': tensor(361)})
Epoch: 1
Train : Counter({'tp': tensor(28268), 'tn': tensor(25790), 'fp': tensor(2260), 'fn': tensor(958)})
Validation : Counter({'tp': tensor(9477), 'tn': tensor(8534), 'fp': tensor(765), 'fn': tensor(316)})
Test : Counter({'tp': tensor(9338), 'tn': tensor(8679), 'fp': tensor(763), 'fn': tensor(313)})
Epoch: 2
Train : Counter({'tp': tensor(27228), 'tn': tensor(26696), 'fn': tensor(1998), 'fp': tensor(1354)})
Validation : Counter({'tp': tensor(9122), 'tn': tensor(8807), 'fn': tensor(671), 'fp': tensor(492)})
Test : Counter({'tp': tensor(9005), 'tn': tensor(8993), 'fn': tensor(646), 'fp': tensor(449)})
"""

In [121]:
from glob import glob

In [122]:
def scores(elt, dataset, epoch) :
    elt = eval(elt)
    acc = (elt['tp'] + elt['tn'])/(sum(elt.values()))
    prec = elt['tp'] / (elt['tp'] + elt['fp'])
    rec = elt['tp'] / (elt['tp'] + elt['fn'])
    f1 = 2 * rec * prec / (prec + rec)
    return {'dataset':dataset, 'epoch':epoch, 'accuracy' : acc, 'precision' : prec, 'recall' : rec, 'f1' : f1}

In [123]:
def gen_scores() :
    for i in range(3) :
        for name in ['Train', 'Validation', 'Test'] :
            yield lambda x : scores(x, name, i)

In [124]:
docs = list(sorted(glob('*.out')))

In [125]:
def compute_res(doc) :
    with open(doc, 'r') as f :
        res = []
        txt = f.read()
        print(txt.split('\n', 1)[0])
        txt = re.sub(r'tensor\((.+?)\)', r'\1', txt)
        scores_funcs = gen_scores()
        for elem in re.findall(r'\{.*?\}', txt) :
            res.append(next(scores_funcs)(elem))
        
        z = pd.DataFrame(res, columns=['dataset', 'epoch', 'accuracy', 'precision', 'recall', 'f1']).set_index(['epoch', 'dataset'])
        return z.unstack()

In [126]:
def bold_formatter(x, value, num_decimals=2):
    """
    Taken from https://stackoverflow.com/questions/65070070/highlight-the-best-value-of-each-row-in-python-pandas-to-latex
    """
    if abs(x - value) < 1e-6 :
        return f"\\textbf\{{{x:.{num_decimals}f}\}}"
    else:
        return f"{x:.{num_decimals}f}"

In [127]:
from functools import partial

In [128]:
print(f'lr = 1e-5')
df = compute_res(docs[0])
df

# fmts = {column: partial(bold_formatter, value=df[column].min(), num_decimals=2) for column in df.columns}
# print(df.to_latex(formatters=fmts))

lr = 1e-5
Epoch: 0


accuracy                      precision                       \
dataset      Test     Train Validation      Test     Train Validation   
epoch                                                                   
0        0.944273  0.942053   0.941232  0.939863  0.939773   0.937355   
1        0.944325  0.942716   0.942070  0.936737  0.936006   0.934307   
2        0.944744  0.943467   0.943013  0.925629  0.925475   0.924096   

           recall                             f1                       
dataset      Test     Train Validation      Test     Train Validation  
epoch                                                                  
0        0.950575  0.947136   0.948841  0.945189  0.943440   0.943063  
1        0.954305  0.952884   0.954151  0.945440  0.944370   0.944124  
2        0.968501  0.967084   0.968447  0.946580  0.945822   0.945752

In [129]:
compute_res(docs[1])

lr : 1.4962356560944327e-05


accuracy                      precision                       \
dataset      Test     Train Validation      Test     Train Validation   
epoch                                                                   
0        0.944797  0.942786   0.943484  0.931880  0.931261   0.931130   
1        0.941864  0.940883   0.941808  0.922362  0.923218   0.923678   
2        0.942963  0.942576   0.941599  0.921441  0.922330   0.920772   

           recall                             f1                       
dataset      Test     Train Validation      Test     Train Validation  
epoch                                                                  
0        0.961040  0.958633   0.960890  0.946235  0.944749   0.945776  
1        0.966325  0.964347   0.966405  0.943832  0.943334   0.944558  
2        0.969848  0.969069   0.969570  0.945025  0.945122   0.944541

In [130]:
compute_res(docs[2])

lr : 2.238721138568338e-05


accuracy                      precision                       \
dataset      Test     Train Validation      Test     Train Validation   
epoch                                                                   
0        0.924527  0.923511   0.920438  0.959584  0.958920   0.956019   
1        0.943644  0.943816   0.943379  0.924463  0.925970   0.925308   
2        0.942649  0.941476   0.939084  0.952507  0.952628   0.948825   

           recall                             f1                       
dataset      Test     Train Validation      Test     Train Validation  
epoch                                                                  
0        0.888094  0.888148   0.885633  0.922456  0.922178   0.919481  
1        0.967568  0.967221   0.967732  0.945525  0.946146   0.946044  
2        0.933064  0.931636   0.931482  0.942685  0.942015   0.940073

In [131]:
compute_res(docs[3])

lr : 3.3496543915782794e-05


accuracy                      precision                       \
dataset      Test     Train Validation      Test     Train Validation   
epoch                                                                   
0        0.942440  0.942367   0.941284  0.925557  0.926216   0.924931   
1        0.940973  0.940534   0.940446  0.916211  0.916210   0.915834   
2        0.941026  0.940272   0.940708  0.916463  0.916411   0.916595   

           recall                             f1                       
dataset      Test     Train Validation      Test     Train Validation  
epoch                                                                  
0        0.963631  0.963834   0.963750  0.944210  0.944650   0.943942  
1        0.972127  0.972388   0.973348  0.943341  0.943463   0.943716  
2        0.971920  0.971566   0.972940  0.943377  0.943183   0.943927

In [134]:
compute_res(docs[4])

lr : 5.011872336272725e-05


accuracy                      precision                       \
dataset      Test     Train Validation      Test     Train Validation   
epoch                                                                   
0        0.942492  0.941948   0.942070  0.921206  0.921937   0.921413   
1        0.938930  0.939346   0.939189  0.911613  0.913196   0.912620   
2        0.941130  0.940027   0.940184  0.920588  0.920995   0.920564   

           recall                             f1                       
dataset      Test     Train Validation      Test     Train Validation  
epoch                                                                  
0        0.969122  0.968213   0.969774  0.944557  0.944508   0.944975  
1        0.973578  0.973688   0.974778  0.941577  0.942472   0.942675  
2        0.966946  0.965271   0.966813  0.943198  0.942613   0.943122